In [19]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time

import warnings
warnings.filterwarnings("ignore")


from sklearn.ensemble import RandomForestRegressor


table = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/starch-sanchez-v3/matrices/A6.pkl")
tableg = table.T

B = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/starch-sanchez-v3/matrices/B6.pkl")
Bdf = pd.DataFrame(B)


tableg = table.T
species_ones = [] 
for i in range(6):
    species = i
    species_ones.append(list(tableg[tableg[species]==1].index))
    
    
species_zeros = []
for i in range(6):
    species = i
    species_zeros.append(list(tableg[tableg[species]==0].index))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       


In [20]:
sp_list_provided = ["C","E","M","P","S","T"]

sstv3 = pd.read_pickle("~/compressed_sensingv1/realdatasets/starch-sanchez-v5/starch-sanchez-relative.pkl")

wellsp = list(sstv3.index)

In [21]:
#table[wellsp]

In [22]:
dataTR = pd.read_pickle("~/compressed_sensingv1/realdatasets/starch-sanchez-v5/traindataRELss.pkl")
dataTE = pd.read_pickle("~/compressed_sensingv1/realdatasets/starch-sanchez-v5/testdataRELss.pkl")

In [25]:
dataTE

,k-fold,fold number,test data,species
0,3,0,"[4, 21, 30, 20, 15, 1, 12]",0
1,3,1,"[29, 24, 5, 28, 13, 23, 2]",0
2,3,2,"[9, 8, 27, 25, 31, 16, 0]",0
3,3,0,"[28, 6, 16, 5, 9, 11, 4, 20, 0]",1
4,3,1,"[30, 15, 29, 10, 24, 8, 3, 13, 23]",1
5,3,2,"[27, 21, 7, 25, 1, 2, 14, 31]",1
6,3,0,"[21, 23, 31, 25, 17, 3, 15, 2, 30]",2
7,3,1,"[10, 14, 5, 29, 7, 1, 6, 0, 27]",2
8,3,2,"[20, 9, 13, 16, 28, 24, 4, 8, 11]",2
9,3,0,"[9, 14, 20, 25, 23, 21, 3, 17, 28]",3


In [5]:
cv_splitsl = [3,5] 
max_depthl = [2,5,10,15,20]
ntrees = [100,200,300,400,500,1000]
max_featuresl = ["auto","sqrt","log2"]

In [6]:
list_present = wellsp
rfper = [] 
for cv_splits in cv_splitsl: 
    datatrain = dataTR[dataTR["k-fold"]==cv_splits]
    datatest = dataTE[dataTE["k-fold"]==cv_splits]
    for speciesc in range(6):
        species = speciesc
        datatousetrain = datatrain[datatrain["species"]==speciesc]
        datatousetest = datatest[datatest["species"]==speciesc]
        df2 = pd.DataFrame(species_ones[speciesc])
        idxlist = list(df2[0].values)
        abunvec = pd.DataFrame(sstv3[sp_list_provided[speciesc]][list(set(wellsp).intersection(species_ones[speciesc]))])
        pp = list(abunvec.index)
        redlist = []
        for kk in pp:
            redlist.append(idxlist.index(kk))
        abunvec.index = redlist
        
        padf = table[species_ones[species]].T
        padf.index = np.arange(0,32,1)
        for md in max_depthl:
                for n_est in ntrees:
                    for mfeatures in max_featuresl:
                        for folds in range(cv_splits):
                            rinew = datatousetrain[datatousetrain["fold number"]==folds]["train data"].values[0]
                            rileft = datatousetest[datatousetest["fold number"]==folds]["test data"].values[0]
                            y3 = abunvec.T[rinew].T[sp_list_provided[speciesc]].values  
                            xtrain = padf.T[rinew].T
                            ytrain = y3
                            
                            regr = RandomForestRegressor(max_depth=md,n_estimators=n_est,max_features=mfeatures)
                            regr.fit(xtrain,ytrain)
                            xtest = padf.T[rileft].T
                            ypred = regr.predict(xtest)
                            rfper.append([speciesc,md,n_est,rinew,rileft,ypred,abunvec.T[rileft].T[sp_list_provided[speciesc]].values,cv_splits,mfeatures])


In [11]:
rdf = pd.DataFrame(rfper,columns=["species","maxdepth","ntrees","Xtrain","Xtest","ypred","yobs","k-fold","features"])

In [12]:
pd.to_pickle(rdf,"RF-REL-per.pkl")